# Connect to OPeNDAP
Here is a tutorial on how to use OPeNDAP to get at data on the THREDDS server at NKN (Univeristy of Idaho): https://climate.northwestknowledge.net/MACA/OPENDAP.php

There are a couple of places on the THREDDS server where there are drought indicators: 
-	[CIRC Drought folder](http://thredds.northwestknowledge.net:8080/thredds/catalog/DROUGHT_LAYERS/catalog.html): VEGDRI, USDM, QUICKDRI, MCDI, LERI, GRADE, FDSI, ESI
-	[GRIDMET DROUGHT indicators](http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_MET_aggregated_catalog.html): SPI/SPEI/EDDI at different timescales (these are in the Drought Stripes tool)
-	VIC DROUGHT indicators: I'm currently making these from the WUSA/CONUS datasets for SRI (standardized runoff index), SM/TM/SWE percentiles
    -	[WUSA](http://thredds.northwestknowledge.net:8080/thredds/catalog/VIC_UCLA/VIC-WUSA/catalog.html)-  there's going to be some new files soon that all the daily data in a single file
    -	[CONUS](http://thredds.northwestknowledge.net:8080/thredds/catalog/VIC_UCLA/VIC-CONUS/catalog.html) - there's going to be some new files soon that all the daily data in a single file

There's also another resource you should be aware of that might help:    
-	[Climate Engine](https://app.climateengine.com/climateEngine)    
    -	USDM raster data over CONUS: there is a rasterized version of the USDM drought classification over CONUS that might be useful to pull a time series from for a point location. I'm looking to put this full raster data on THREDDS at some point soon.. just need some time to aggregate the files. 
    -	Standardized NDVI or  percentiles in NDVI from remote sensing (MODIS or Landsat) might be useful here for vegetation drought.


In [12]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import xarray as xr
from netCDF4 import Dataset

dirPath = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC'

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
file = 'DROUGHT_LAYERS/VEGDRI/vegdri_20201011.nc'
xr.open_dataset(f"{dirPath}/{file}")

<xarray.Dataset>
Dimensions:  (lat: 2687, lon: 5791)
Coordinates:
  * lat      (lat) float64 51.77 51.75 51.74 51.73 ... 22.51 22.5 22.49 22.48
  * lon      (lon) float64 -128.5 -128.5 -128.5 -128.5 ... -65.42 -65.41 -65.4
Data variables:
    vegdri   (lat, lon) float64 ...